In [2]:
!pip install datasets
!pip install evaluate
!pip install transformers[torch]
!pip install git-python
!pip install secrebleu
!pip install rouge_score
!pip install dataclasses
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement secrebleu (from versions: none)
ERROR: No matching distribution found for secrebleu
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=b449bb335146018789a28211078311a02f662cbe477339a333cde2f11fd223e1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import datasets
import transformers
from transformers import BertTokenizerFast, BertTokenizer
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional
from datasets import Dataset
# from se2seq_trainer

import numpy as np
import pandas as pd
import torch

In [4]:
c_train = pd.read_csv("/content/drive/MyDrive/datasets_liputan6/id_liputan6/c_train_df_100k.csv")
c_test = pd.read_csv("/content/drive/MyDrive/datasets_liputan6/id_liputan6/c_test_df.csv")
c_dev = pd.read_csv("/content/drive/MyDrive/datasets_liputan6/id_liputan6/c_dev_df.csv")

x_test = pd.read_csv("/content/drive/MyDrive/datasets_liputan6/id_liputan6/xt_test_df.csv")
x_dev = pd.read_csv("/content/drive/MyDrive/datasets_liputan6/id_liputan6/xt_dev_df.csv")

c_train_50k = pd.read_csv("/content/drive/MyDrive/datasets_liputan6/id_liputan6/c_train_df_5k.csv")

In [5]:
c_train

,Unnamed: 0,clean_article,clean_summary
0,0,"Liputan6. com, Jakarta : Setelah menjalani Uji...",Artis muda Haykal dan Putri Titan akan memasuk...
1,1,Striker Timnas Inggris yang bermain bersama To...,Pertandingan melawan Manchester City merupakan...
2,2,"Liputan6. com, Jakarta : Harga sejumlah kebutu...","Harga bahan pokok seperti minyak goreng, telur..."
3,3,"Liputan6, com, Kupang : Evakuasi warga negara ...",Evakuasi warga negara Indonesia dari Timor Les...
4,4,"Liputan6. com, Lembata : Pusat Vulkanologi dan...",Status Gunung Batutara ditetapkan siaga tiga k...
...,...,...,...
193878,193878,"Liputan6. com, Padang : Sebanyak 17. 928. 925 ...",Jumlah peserta Jamsostek nonaktif jauh lebih b...
193879,193879,"Liputan6. com, Jakarta : Setelah sepekan berla...",Razia pekerja seks komersial yang digelar Satp...
193880,193880,"Liputan6. com, Bandung : Mantan Rektor Institu...",I Nyoman Sumaryadi kembali diperiksa di Mapold...
193881,193881,"Peringatan 40 Hari Pascagempa color= "" ffff00 ...",Peringatan 40 hari pascagempa bumi Yogyakarta ...


In [6]:
dataset_train = Dataset.from_dict(c_train)
dataset_test = Dataset.from_dict(c_test)
dataset_eval = Dataset.from_dict(c_dev)

dataset_train50k = Dataset.from_dict(c_train_50k)

In [7]:
model_tokenizer = "indolem/indobert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_tokenizer)
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

In [8]:
batch_size=4  # change to 16 for full training
encoder_max_length=512
decoder_max_length=128

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["clean_article"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["clean_summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch



In [9]:
# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
train_data = dataset_train

train_data = train_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["clean_article", "clean_summary", "Unnamed: 0"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
val_data = dataset_eval

val_data = val_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["clean_article", "clean_summary", "Unnamed: 0"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
test_data = dataset_test

test_data = test_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["clean_article", "clean_summary", "Unnamed: 0"]
)
test_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Map:   0%|          | 0/193883 [00:00<?, ? examples/s]

Map:   0%|          | 0/10972 [00:00<?, ? examples/s]

Map:   0%|          | 0/10972 [00:00<?, ? examples/s]

In [ ]:
# columns = ['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels']

# train_data = train_data.set_format(columns=columns)
# test_data = test_data.set_format(columns=columns)
# val_data = val_data.set_format(columns=columns)

# train_data

In [10]:
print(train_data)

Dataset({
    features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
    num_rows: 193883
})


In [11]:
model_name = "gaunernst/bert-tiny-uncased" 

bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name)

config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertLMHeadModel were not initialized from the model checkpoint at gaunernst/bert-tiny-uncased and are newly initialized: ['bert.encoder.layer.1.crossattention.self.query.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.1.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.value.weight',

In [ ]:
CUDA_LAUNCH_BLOCKING=1

In [12]:
bert2bert.config.decoder_start_token_id = tokenizer.bos_token_id
bert2bert.config.eos_token_id = tokenizer.eos_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id

bert2bert.encoder.resize_token_embeddings(1000)
bert2bert.decoder.resize_token_embeddings(1000)
bert2bert.config.num_layers = 4
bert2bert.config.hidden_size = 256
bert2bert.config.num_hidden_layers = 4
bert2bert.config.num_attention_heads = 4
bert2bert.config.vocab_size =  1000 # bert2bert.config.decoder.vocab_size
bert2bert.config.max_length = 128
bert2bert.config.min_length = 20
bert2bert.config.no_repeat_ngram_size = 3
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 4

In [13]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

<ipython-input-13-4324511b13f7>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = datasets.load_metric("rouge")


In [ ]:
@dataclass
class CustomSeq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
    )
    sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
    )
    adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
    )
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
    )
    dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
    )
    lr_scheduler: Optional[str] = field(
        default="linear", metadata={"help": f"Which lr scheduler to use."}
    )

In [14]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Summarization_Model/Bert2Bert_summ_liputan6",
    # per_device_train_batch_size=batch_size,
    # per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    overwrite_output_dir=True,
    do_predict=True,
    # evaluate_during_training=True,
    do_train=True,
    do_eval=True,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=10,
    save_steps=1000,
    eval_steps=500,
    warmup_steps=500,
    # max_steps=16, # delete for full training
    save_total_limit=3,
    gradient_accumulation_steps= 2,
    learning_rate= 2e-5,
    weight_decay= 0.01,
    # fp16=True,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bert2bert,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
)


In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [18]:
trainer.train(resume_from_checkpoint="/content/drive/MyDrive/Summarization_Model/Bert2Bert_summ_liputan6/checkpoint-48000")

There were missing keys in the checkpoint model loaded: ['decoder.cls.predictions.decoder.weight', 'decoder.cls.predictions.decoder.bias'].
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:639: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss
48010,0.000200
48020,0.000300
48030,0.002300
48040,0.000200
48050,0.000200
48060,0.000700
48070,0.000200
48080,0.001400
48090,0.001000
48100,0.001800


TrainOutput(global_step=48470, training_loss=9.371993336665161e-06, metrics={'train_runtime': 29.3933, 'train_samples_per_second': 13192.314, 'train_steps_per_second': 1649.014, 'total_flos': 1181122840538112.0, 'train_loss': 9.371993336665161e-06, 'epoch': 2.0})

In [ ]:
!nvidia-smi

In [ ]:
trainer.evaluate()

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name)
model = EncoderDecoderModel.from_pretrained("/content/checkpoint-16")
model.to("cuda")

batch_size = 16

test_data = dataset_test.select(range(16))

def generate_summary(batch):

  inputs = tokenizer(batch['clean_article'], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
  input_ids = inputs.input_ids.to("cuda")
  attention_mask = inputs.attention_mask.to("cuda")

  outputs = model.generate(input_ids, attention_mask=attention_mask)
  outputs_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

  batch['pred'] = outputs_str

  return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["clean_article"])

pred_str = results['pred']
label_str = results['clean_summary']

rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output)

In [ ]:
import re
from transformers import pipeline

# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = EncoderDecoderModel.from_pretrained("/content/checkpoint-16")
# model.to("cuda")
# summerizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0, framework='pt')

# result = summerizer(c_test['clean_article'][1],)

# def post_process_summary(summary):
#     # Remove consecutive repeated characters
#     summary = re.sub(r'(.)\1+', r'\1', summary)
#     return summary


# processed_summary = post_process_summary(result[0]['summary_text'])
# print(processed_summary)

In [ ]:
rouge_output = rouge.compute(predictions=result, references=test_data['clean_summary'][0], rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output)

In [ ]:
test_data['clean_summary'][0]